In [ ]:
# !pip install -U accelerate
# !pip install -U transformers

import accelerate
import transformers
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, BitsAndBytesConfig, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm.notebook import tqdm
pd.options.display.max_colwidth = 1000

## Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2024-05-23 05:00:54.445955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 05:00:54.446088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 05:00:54.575077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


In [ ]:
# Load model directly'
model_name = "gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [ ]:
train_data = pd.read_csv('0523_942292row.csv')

/tmp/ipykernel_34/1464057813.py:1: DtypeWarning: Columns (0,2,4,5,6,7,8,9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/kaggle/input/0523-train2/0523_942292row.csv')


In [ ]:
## 제주어 토큰, 표준어 토큰 정의하기
jeju_token = "[제주]"
standard_token = "[표준]"

## 양방향 데이터 리스트 생성
bidirectional_data = []

for dialect, standard in zip(train_data['dialect_form'], train_data['standard_form']):
    ## 토큰이 [제주] 일 경우 제주어 -> 표준어
    bidirectional_data.append({
        "source": jeju_token + " " + dialect,
        "target": standard
    })
    ## 토큰이 [표준] 일 경우 표준어 -> 제주어
    bidirectional_data.append({
        "source": standard_token + " " + standard,
        "target": dialect
    })

## 데이터 토크나이징
tokenized_data = []
for item in bidirectional_data:
    source_encodings = tokenizer(item['source'], max_length=64, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(item['target'], max_length=64, truncation=True, padding="max_length", return_tensors="pt")
    tokenized_data.append({
        "input_ids": source_encodings["input_ids"],
        "attention_mask": source_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    })

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# !pip install datasets
from datasets import Dataset, DatasetDict
import pandas as pd

# Modify the DataFrame creation code
## 요렇게 바꾸니까 메모리를 너무너무 아낄 수 있네 !!
formatted_data_df = pd.DataFrame([{
    "input_ids": np.array(fd["input_ids"].numpy().tolist()[0], dtype=np.uint16),
    "attention_mask": np.array(fd["attention_mask"].numpy().tolist()[0], dtype=np.uint8),
    "labels": np.array(fd["labels"].numpy().tolist()[0], dtype=np.uint16)
} for fd in tokenized_data])

## 데이터를 Dataset 형식으로 변환
train_dataset = Dataset.from_pandas(formatted_data_df)

## 데이터셋 한번 확인해보슈
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1884584
})

In [ ]:
## 학습 데이터셋을 학습 및 평가용으로 분리 (예: 90% 학습, 10% 평가)
train_test_split = train_dataset.train_test_split(test_size=0.1)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

## 학습 매개변수 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints",                           ## 결과물을 저장할 디렉토리
    evaluation_strategy="epoch",                         ## 평가 전략
    learning_rate=2e-5,                                  ## 학습률
    per_device_train_batch_size=32,                      ## 디바이스 당 배치 크기
    weight_decay=0.01,                                   ## 가중치 감소
    save_total_limit=3,                                  ## 저장할 최대 체크포인트 수
    num_train_epochs=3,                                  ## 학습 에폭 수
    predict_with_generate=True,                          ## 생성을 사용한 예측 활성화
)

In [ ]:
import wandb

## API 키를 직접 입력

wandb.login(key="")

## 학습 준비
trainer = Seq2SeqTrainer(
    model=model,                         ## 학습할 모델
    args=training_args,                  ## 학습 설정
    train_dataset=dataset_dict['train'], ## 학습 데이터셋
    eval_dataset=dataset_dict['test'],   ## 평가 데이터셋
    tokenizer=tokenizer
)

## 학습 시작
trainer.train()

## 학습 종료
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 